# Versiune a codului din 01.12.2025 în care am integrat prima versiune de vizualizare a caracteristicilor de DD, risk, profit

In [1]:
# ==== Metrici de performanță similare PineScript ====
import numpy as np
import pandas as pd
from typing import Dict, Optional

def _extract_trade_returns(bt: pd.DataFrame, log_returns: bool = True):
    pos = bt.get("position", pd.Series(dtype=float)).fillna(0.0)
    strat_ret = bt.get("strategy", pd.Series(dtype=float)).fillna(0.0)
    trades = []
    in_trade = False
    prev_pos = pos.shift(1).fillna(0.0)
    agg = 0.0 if log_returns else 1.0

    for r, p, prev in zip(strat_ret, pos, prev_pos):
        opened = (prev == 0) and (p != 0)
        closed = (p == 0) and (prev != 0)

        if opened:
            agg = 0.0 if log_returns else 1.0
            in_trade = True

        if in_trade:
            if log_returns:
                agg += r
            else:
                agg *= (1.0 + r)

        if in_trade and closed:
            trades.append(agg if log_returns else agg - 1.0)
            in_trade = False

    return trades

def compute_performance_metrics(bt: pd.DataFrame,
                                initial_equity: float = 1.0,
                                log_returns: bool = True) -> dict:
    if bt is None or bt.empty:
        return {
            "total_pnl": 0.0,
            "total_pnl_pct": 0.0,
            "max_drawdown_pct": 0.0,
            "profit_factor": 0.0,
            "win_rate_pct": 0.0,
            "total_trades": 0,
            "max_trade_duration_days": 0.0,
        }

    equity_curve = bt.get("equity_curve")
    if equity_curve is None or equity_curve.isna().all():
        equity_curve = bt.get("cstrategy")
    equity_curve = equity_curve.astype(float)

    total_equity = float(equity_curve.iloc[-1])
    total_pnl = total_equity - initial_equity
    total_pnl_pct = (total_equity / initial_equity - 1.0) * 100.0

    running_max = equity_curve.cummax()
    dd = (running_max - equity_curve) / running_max.replace(0, np.nan)
    max_dd_pct = float(dd.max() * 100.0) if len(dd) else 0.0

    trade_logs = _extract_trade_returns(bt, log_returns=log_returns)
    trade_pnls = [np.exp(x) - 1.0 for x in trade_logs] if log_returns else list(trade_logs)

    total_trades = len(trade_pnls)
    gross_profit = sum(v for v in trade_pnls if v > 0)
    gross_loss = sum(v for v in trade_pnls if v < 0)
    profit_factor = (
        gross_profit / abs(gross_loss)
        if gross_loss < 0
        else (float("inf") if total_trades else 0.0)
    )

    wins = sum(1 for v in trade_pnls if v > 0)
    win_rate_pct = (wins / total_trades * 100.0) if total_trades else 0.0

    # Durata tranzacțiilor
    pos = bt.get("position", pd.Series(dtype=float)).fillna(0.0)
    idx = bt.index
    durations = []
    start_idx = None
    for cur, prev, ts in zip(pos, pos.shift(1).fillna(0.0), idx):
        if prev == 0 and cur != 0:
            start_idx = ts
        if prev != 0 and cur == 0 and start_idx is not None:
            dur_days = (ts - start_idx).total_seconds() / 86400 if hasattr(ts, "to_pydatetime") else 0.0
            durations.append(dur_days)
            start_idx = None
    max_dur = max(durations) if durations else 0.0

    return {
        "total_pnl": total_pnl,
        "total_pnl_pct": total_pnl_pct,
        "max_drawdown_pct": max_dd_pct,
        "profit_factor": profit_factor,
        "win_rate_pct": win_rate_pct,
        "total_trades": total_trades,
        "max_trade_duration_days": max_dur,
    }


def print_performance_summary(symbol: str, metrics: dict):
    print(f"Rezumat performanță pentru {symbol}:")
    print(f"  Total P&L: {metrics['total_pnl']:.2f} ({metrics['total_pnl_pct']:.2f}%)")
    print(f"  Max drawdown: {metrics['max_drawdown_pct']:.2f}%")
    print(f"  Profit factor: {metrics['profit_factor']:.2f}")
    print(f"  Win rate: {metrics['win_rate_pct']:.2f}% din {metrics['total_trades']} tranzacții")
    print(f"  Durată maximă tranzacție: {metrics['max_trade_duration_days']:.2f} zile")


In [ ]:
# Cod Strategia 1 optimizat + obiectiv cu risk constraints
# Pentru rulare ca script Python.

import numpy as np
import pandas as pd
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass
from typing import Dict, Any, Optional
from scipy.optimize import minimize
from pathlib import Path

# ======================================================
# ============== Metrici de performanță ================
# ======================================================


# ==== Metrici de performanță similare PineScript ====
import numpy as np
import pandas as pd
from typing import Dict, Optional

# ==== Metrici de performanță similare PineScript ====
import numpy as np
import pandas as pd
from typing import Dict, Optional

def _extract_trade_returns(bt: pd.DataFrame, log_returns: bool = True):
    pos = bt.get("position", pd.Series(dtype=float)).fillna(0.0)
    strat_ret = bt.get("strategy", pd.Series(dtype=float)).fillna(0.0)
    trades = []
    in_trade = False
    prev_pos = pos.shift(1).fillna(0.0)
    agg = 0.0 if log_returns else 1.0

    for r, p, prev in zip(strat_ret, pos, prev_pos):
        opened = (prev == 0) and (p != 0)
        closed = (p == 0) and (prev != 0)

        if opened:
            agg = 0.0 if log_returns else 1.0
            in_trade = True

        if in_trade:
            if log_returns:
                agg += r
            else:
                agg *= (1.0 + r)

        if in_trade and closed:
            trades.append(agg if log_returns else agg - 1.0)
            in_trade = False

    return trades

def compute_performance_metrics(bt: pd.DataFrame,
                                initial_equity: float = 1.0,
                                log_returns: bool = True) -> dict:
    if bt is None or bt.empty:
        return {
            "total_pnl": 0.0,
            "total_pnl_pct": 0.0,
            "max_drawdown_pct": 0.0,
            "profit_factor": 0.0,
            "win_rate_pct": 0.0,
            "total_trades": 0,
            "max_trade_duration_days": 0.0,
        }

    equity_curve = bt.get("equity_curve")
    if equity_curve is None or equity_curve.isna().all():
        equity_curve = bt.get("cstrategy")
    equity_curve = equity_curve.astype(float)

    total_equity = float(equity_curve.iloc[-1])
    total_pnl = total_equity - initial_equity
    total_pnl_pct = (total_equity / initial_equity - 1.0) * 100.0

    running_max = equity_curve.cummax()
    dd = (running_max - equity_curve) / running_max.replace(0, np.nan)
    max_dd_pct = float(dd.max() * 100.0) if len(dd) else 0.0

    trade_logs = _extract_trade_returns(bt, log_returns=log_returns)
    trade_pnls = [np.exp(x) - 1.0 for x in trade_logs] if log_returns else list(trade_logs)

    total_trades = len(trade_pnls)
    gross_profit = sum(v for v in trade_pnls if v > 0)
    gross_loss = sum(v for v in trade_pnls if v < 0)
    profit_factor = (
        gross_profit / abs(gross_loss)
        if gross_loss < 0
        else (float("inf") if total_trades else 0.0)
    )

    wins = sum(1 for v in trade_pnls if v > 0)
    win_rate_pct = (wins / total_trades * 100.0) if total_trades else 0.0

    # Durata tranzacțiilor
    pos = bt.get("position", pd.Series(dtype=float)).fillna(0.0)
    idx = bt.index
    durations = []
    start_idx = None
    for cur, prev, ts in zip(pos, pos.shift(1).fillna(0.0), idx):
        if prev == 0 and cur != 0:
            start_idx = ts
        if prev != 0 and cur == 0 and start_idx is not None:
            dur_days = (ts - start_idx).total_seconds() / 86400 if hasattr(ts, "to_pydatetime") else 0.0
            durations.append(dur_days)
            start_idx = None
    max_dur = max(durations) if durations else 0.0

    return {
        "total_pnl": total_pnl,
        "total_pnl_pct": total_pnl_pct,
        "max_drawdown_pct": max_dd_pct,
        "profit_factor": profit_factor,
        "win_rate_pct": win_rate_pct,
        "total_trades": total_trades,
        "max_trade_duration_days": max_dur,
    }


def print_performance_summary(symbol: str, metrics: dict):
    print(f"Rezumat performanță pentru {symbol}:")
    print(f"  Total P&L: {metrics['total_pnl']:.2f} ({metrics['total_pnl_pct']:.2f}%)")
    print(f"  Max drawdown: {metrics['max_drawdown_pct']:.2f}%")
    print(f"  Profit factor: {metrics['profit_factor']:.2f}")
    print(f"  Win rate: {metrics['win_rate_pct']:.2f}% din {metrics['total_trades']} tranzacții")
    print(f"  Durată maximă tranzacție: {metrics['max_trade_duration_days']:.2f} zile")
# ======================================================
# ========= Pregătire vectori de optimizare ============
# ======================================================

def prepare_optimization_vectors():
    global indicator_params_default, short_params_default

    if "indicator_params_default" not in globals():
        indicator_params_default = {
            "fEma_Length": 58, "sEma_Length": 418,
            "ADX_len": 13, "ADX_smo": 13, "th": 6.26,
            "fastLength": 20, "slowLength": 42, "signalLength": 12,  # MACD clasic
            "MACD_options": "MACD",
            "BB_Length": 100, "BB_mult": 10,
            "sma_Length": 81, "volume_f": 0.87,
            "DClength": 82,
            "Sst": 0.10, "Sinc": 0.04, "Smax": 0.40,
            "bbMinWidth01": 9.3,
            "lengthz": 14, "lengthStdev": 16, "A": -0.1, "B": 0.5, "bbMinWidth02": 0.0,
        }

    if "short_params_default" not in globals():
        short_params_default = {
            "tp": 3.6, "sl": 5.0,
            "atrPeriodSl": 116, "multiplierPeriodSl": 107, "trailOffset": 0.38,
            "reverse_exit": True, "exit_on_entry_loss": False,
            "Position": "Short",
        }

    # permite alegerea modului de Stop Loss din variabila globală SL_MODE
    sl_override = globals().get("SL_MODE")
    if sl_override:
        short_params_default["SL_options"] = str(sl_override)

    # deocamdată optimizăm DOAR parametrii de SL (poți activa și indicatorii, dacă vrei)
    indicator_params_to_optimize = [
        # "fEma_Length", "sEma_Length",
        # "ADX_len", "ADX_smo", "th",
        # "fastLength", "slowLength", "signalLength",
        # "BB_Length", "BB_mult",
        # "DClength",
    ]

    sl_mode = str(short_params_default.get("SL_options", "Both"))
    short_params_to_optimize = []
    if sl_mode in ("Both", "ATR"):
        short_params_to_optimize.extend(["atrPeriodSl", "multiplierPeriodSl"])
    if sl_mode in ("Both", "Normal"):
        short_params_to_optimize.append("sl")

    MACZ_KEYS = {"MACD_options", "lengthz", "lengthStdev", "A", "B"}
    names = [n for n in (indicator_params_to_optimize + short_params_to_optimize) if n not in MACZ_KEYS]

    LENGTH_INTS = {
        "fEma_Length", "sEma_Length", "ADX_len", "ADX_smo",
        "fastLength", "slowLength", "signalLength",
        "BB_Length", "sma_Length", "DClength", "atrPeriodSl",
    }
    SAR_FLOATS = {"Sst", "Sinc", "Smax"}
    PCT_FLOATS = {"tp", "sl", "bbMinWidth01"}
    POS_FLOATS = {"BB_mult", "volume_f", "th", "multiplierPeriodSl"}
    SHORT_FLOATS = {"trailOffset"}

    # extinderi unilaterale acolo unde s-au lovit capetele
    BOUND_OVERRIDES = {
        "ADX_len": (0.48, 1.17),
        "ADX_smo": (0.40, 1.40),
        "th": (0.33, 2.50),
        "fEma_Length": (0.83, 1.30),
        "BB_Length": (0.83, 2.20),
        "BB_mult": (0.50, 2.50),
        "DClength": (0.80, 1.40),
        "atrPeriodSl": (0.50, 1.50),
        "multiplierPeriodSl": (0.30, 1.2),
        "sl": (0.25, 1.20),
    }

    start_params, bounds = [], []

    for name in names:
        val = float(indicator_params_default.get(name, short_params_default.get(name)))

        if name in LENGTH_INTS:
            lo_mult, hi_mult = BOUND_OVERRIDES.get(name, (0.83, 1.17))
            lo = max(2, int(round(val * lo_mult)))
            hi = max(lo + 1, int(round(val * hi_mult)))
            start_params.append(int(round(val)))
            bounds.append((lo, hi))

        elif name in SAR_FLOATS:
            lo_mult, hi_mult = BOUND_OVERRIDES.get(name, (0.50, 2.00))
            lo = val * lo_mult
            hi = val * hi_mult
            start_params.append(val)
            bounds.append((lo, hi))

        else:
            lo_mult, hi_mult = BOUND_OVERRIDES.get(name, (0.50, 1.50))
            lo = val * lo_mult
            hi = val * hi_mult
            start_params.append(val)
            bounds.append((lo, hi))

    return names, tuple(start_params), tuple(bounds)


def vector_to_param_dicts(x, names):
    """Mapează vectorul x -> (indicator_params, short_params)."""
    indicator_params = indicator_params_default.copy()
    short_params = short_params_default.copy()

    int_params = {
        "fEma_Length", "sEma_Length", "ADX_len", "ADX_smo",
        "fastLength", "slowLength", "signalLength",
        "sma_Length", "BB_Length", "DClength", "atrPeriodSl",
    }
    QUANTIZE = {
        "th": 0.1,
        "BB_mult": 0.1,
        "multiplierPeriodSl": 0.25,
    }

    for name, v in zip(names, x):
        if name in int_params:
            val = max(1, int(round(v)))
        else:
            val = float(v)
            if name in QUANTIZE:
                step = QUANTIZE[name]
                val = round(val / step) * step

        if name in indicator_params:
            indicator_params[name] = val
        elif name in short_params:
            short_params[name] = val

    indicator_params["MACD_options"] = "MACD"
    for k in ("lengthz", "lengthStdev", "A", "B"):
        indicator_params.pop(k, None)

    return indicator_params, short_params


def _get_default(name: str) -> Any:
    return indicator_params_default.get(name, short_params_default.get(name))

# inițializăm o singură dată
names, start_params, bounds = prepare_optimization_vectors()
print("Optimizez", len(names), "parametri:", names)

# cache global CSV-uri
all_data: Dict[str, pd.DataFrame] = {}

# ======================================================
# ===== Helperi pentru date, spread, validare ==========
# ======================================================

DATA_DIR = os.environ.get("SUPER8_DATA_DIR", "Simboluri_Binance")


def symbol_csv_path(symbol: str) -> str:
    return os.path.join(DATA_DIR, f"{symbol}.csv")


def symbol_csv_path_case_insensitive(symbol: str) -> Optional[str]:
    p1 = Path(symbol_csv_path(symbol)).resolve()
    p2 = Path(DATA_DIR, f"{symbol}.CSV").resolve()
    if p1.exists():
        return str(p1)
    if p2.exists():
        return str(p2)
    return None


def _crossover(a: pd.Series, b: pd.Series) -> pd.Series:
    a1 = a.shift(1)
    b1 = b.shift(1)
    return (a1 <= b1) & (a > b)


def _crossunder(a: pd.Series, b: pd.Series) -> pd.Series:
    a1 = a.shift(1)
    b1 = b.shift(1)
    return (a1 >= b1) & (a < b)


def _rma(series: pd.Series, period: int) -> pd.Series:
    alpha = 1.0 / period
    return series.ewm(alpha=alpha, adjust=False).mean()

USE_BNB_FEES = True  # True: 0.036% taker; False: 0.04% taker

def _ensure_price(df: pd.DataFrame) -> pd.DataFrame:
    if "Price" not in df.columns:
        if "close" in df.columns:
            df["Price"] = df["close"]
        else:
            raise KeyError("CSV must contain 'Price' or 'close'.")
    for col in ("high", "low"):
        if col not in df.columns:
            raise KeyError(f"CSV must contain '{col}'.")
    return df

def _k_from_barlen(bar_length: str) -> float:
    bl = (bar_length or "").upper()
    # monoton crescător cu timeframe-ul
    if bl in ("M1", "1M"):   return 0.08
    if bl in ("M5", "5M"):   return 0.12
    if bl in ("M15", "15M"): return 0.18
    if bl in ("M30", "30M"): return 0.20
    if bl in ("H1", "1H"):   return 0.22
    if bl in ("H2", "2H"):   return 0.26
    if bl in ("H4", "4H"):   return 0.30
    if bl in ("H6", "6H"):   return 0.32
    if bl in ("H12","12H"):  return 0.35
    if bl in ("D1", "1D"):   return 0.40
    return 0.26  # default H2

def compute_spread(
    df: pd.DataFrame,
    pay_fees_with_bnb: bool = USE_BNB_FEES,
    bar_length: str = "H2",
    slip_floor: float = 0.00005,  # 0.005% (mai realist decât 0.02%)
    slip_cap: float = 0.0040      # 0.40%
) -> float:
    """
    Spread = cost per-side în LOG-return.
    fee_pct: Binance Futures taker 0.04% (sau 0.036% cu BNB)
    slip_pct: k * median((high-low)/Price) cu clip [slip_floor, slip_cap]
    """
    df = _ensure_price(df)

    fee_pct = 0.00036 if pay_fees_with_bnb else 0.0004

    rng = (df["high"] - df["low"]) / df["Price"]
    k = _k_from_barlen(bar_length)
    slip_pct = float(np.clip(k * float(np.nanmedian(rng)), slip_floor, slip_cap))

    spread_log = float(-np.log(1.0 - (fee_pct + slip_pct)))
    df["Spread"] = spread_log
    return spread_log



def is_param_combo_valid(ind_params: Dict[str, Any], short_vals: Dict[str, Any]) -> bool:
    if int(ind_params["fEma_Length"]) >= int(ind_params["sEma_Length"]):
        return False
    if int(ind_params["fastLength"]) >= int(ind_params["slowLength"]):
        return False
    if int(ind_params.get("ADX_smo", ind_params["ADX_len"])) > int(ind_params["ADX_len"]):
        return False
    if not (ind_params["Sst"] > 0 and ind_params["Sinc"] > 0 and ind_params["Smax"] > 0):
        return False
    if ind_params["Sst"] > ind_params["Smax"] or ind_params["Sinc"] > ind_params["Smax"]:
        return False
    if int(ind_params["BB_Length"]) < 5:
        return False
    if float(ind_params["BB_mult"]) <= 0:
        return False
    if int(ind_params["DClength"]) < 5:
        return False
    if int(ind_params["sma_Length"]) < 1:
        return False
    if float(ind_params["volume_f"]) <= 0:
        return False
    if int(short_vals["atrPeriodSl"]) < 1:
        return False
    if float(short_vals["multiplierPeriodSl"]) <= 0:
        return False
    if float(short_vals["trailOffset"]) < 0:
        return False
    if float(short_vals["tp"]) < 0 or float(short_vals["sl"]) < 0:
        return False
    return True


def why_invalid(ind_params, short_vals):
    reasons = []
    if int(ind_params["fEma_Length"]) >= int(ind_params["sEma_Length"]):
        reasons.append("fEma>=sEma")
    if int(ind_params["fastLength"]) >= int(ind_params["slowLength"]):
        reasons.append("fast>=slow")
    if int(ind_params["ADX_smo"]) > int(ind_params["ADX_len"]):
        reasons.append("ADX_smo>ADX_len")
    if not (ind_params["Sst"] > 0 and ind_params["Sinc"] > 0 and ind_params["Smax"] > 0):
        reasons.append("SAR<=0")
    if ind_params["Sst"] > ind_params["Smax"] or ind_params["Sinc"] > ind_params["Smax"]:
        reasons.append("SAR start/inc > max")
    if int(ind_params["BB_Length"]) < 5 or float(ind_params["BB_mult"]) <= 0:
        reasons.append("BB invalid")
    if int(ind_params["DClength"]) < 5:
        reasons.append("DC<5")
    if int(ind_params["sma_Length"]) < 1 or float(ind_params["volume_f"]) <= 0:
        reasons.append("volume invalid")
    if int(short_vals["atrPeriodSl"]) < 1 or float(short_vals["multiplierPeriodSl"]) <= 0 or float(short_vals["trailOffset"]) < 0:
        reasons.append("SL invalid")
    if float(short_vals["tp"]) < 0 or float(short_vals["sl"]) < 0:
        reasons.append("tp/sl<0")
    return reasons

# ======================================================
# =============== Indicatori Super 8 ===================
# ======================================================

class Super8Indicators:
    def __init__(self, params: Dict[str, Any]):
        self.p = params

    def _ema(self, s: pd.Series, span: int) -> pd.Series:
        return s.ewm(span=span, adjust=False).mean()

    def _adx_block(self, df: pd.DataFrame, length: int, smoothing: int, threshold: float) -> pd.DataFrame:
        tr = np.maximum(
            df["high"] - df["low"],
            np.maximum((df["high"] - df["Price"].shift(1)).abs(),
                       (df["low"] - df["Price"].shift(1)).abs()),
        )
        up = df["high"].diff()
        dn = -df["low"].diff()
        plus_dm = np.where((up > dn) & (up > 0), up, 0.0)
        minus_dm = np.where((dn > up) & (dn > 0), dn, 0.0)

        tr_s = _rma(pd.Series(tr, index=df.index), length)
        plus_s = _rma(pd.Series(plus_dm, index=df.index), length)
        minus_s = _rma(pd.Series(minus_dm, index=df.index), length)

        plus_di = (plus_s / tr_s) * 100.0
        minus_di = (minus_s / tr_s) * 100.0
        denom = (plus_di + minus_di).clip(lower=1e-10)
        dx = ((plus_di - minus_di).abs() / denom) * 100.0
        adx = _rma(dx, smoothing)

        out = pd.DataFrame(index=df.index)
        out["adx"] = adx
        out["di_plus"] = plus_di
        out["di_minus"] = minus_di
        out["ADX_longCond"] = (plus_di > minus_di) & (adx > threshold)
        out["ADX_shortCond"] = (plus_di < minus_di) & (adx > threshold)
        return out

    def _sar_tv(self, high: pd.Series, low: pd.Series,
                start: float, step: float, smax: float,
                price: pd.Series) -> pd.Series:
        h, l = high.to_numpy(), low.to_numpy()
        n = len(h)
        if n == 0:
            return pd.Series([], index=high.index, dtype=float)

        psar = np.zeros(n, dtype=float)
        up = True
        if n >= 2:
            if not (pd.isna(price.iloc[0]) or pd.isna(price.iloc[1])):
                up = bool(price.iloc[1] >= price.iloc[0])
            else:
                up = bool((h[1] + l[1]) >= (h[0] + l[0]))

        af = start
        ep = float(h[0] if up else l[0])
        psar[0] = float(l[0] if up else h[0])

        for i in range(1, n):
            psar[i] = psar[i - 1] + af * (ep - psar[i - 1])
            if up:
                if i >= 2:
                    psar[i] = min(psar[i], l[i - 1], l[i - 2])
                else:
                    psar[i] = min(psar[i], l[i - 1])
                if h[i] > ep:
                    ep = h[i]
                    af = min(af + step, smax)
                if l[i] < psar[i]:
                    up = False
                    psar[i] = ep
                    ep = l[i]
                    af = start
            else:
                if i >= 2:
                    psar[i] = max(psar[i], h[i - 1], h[i - 2])
                else:
                    psar[i] = max(psar[i], h[i - 1])
                if l[i] < ep:
                    ep = l[i]
                    af = min(af + step, smax)
                if h[i] > psar[i]:
                    up = True
                    psar[i] = ep
                    ep = h[i]
                    af = start

        return pd.Series(psar, index=high.index, name="SAR")

    def _macd(self, s: pd.Series, fast: int, slow: int, signal: int) -> pd.DataFrame:
        fast_ma = s.ewm(span=fast, adjust=False).mean()
        slow_ma = s.ewm(span=slow, adjust=False).mean()
        macd_line = fast_ma - slow_ma
        signal_line = macd_line.ewm(span=signal, adjust=False).mean()
        hist = macd_line - signal_line
        out = pd.DataFrame(index=s.index)
        out["hist"] = hist
        out["lMACD"] = macd_line
        out["sMACD"] = signal_line
        return out

    def _macz(self, df: pd.DataFrame) -> pd.DataFrame:
        lengthz = self.p["lengthz"]
        lengthStdev = self.p["lengthStdev"]
        A = self.p["A"]
        B = self.p["B"]
        signalLength = self.p["signalLength"]

        vol = df["volume"]
        px = df["Price"]
        vw_mean = (vol * px).rolling(window=lengthz, min_periods=lengthz).sum() / \
                  vol.rolling(window=lengthz, min_periods=lengthz).sum()
        vw_sd = (px - vw_mean).pow(2).rolling(window=lengthz, min_periods=lengthz).mean().pow(0.5)
        zscore = (px - vw_mean) / vw_sd
        macd_std = px.rolling(window=lengthStdev, min_periods=lengthStdev).std(ddof=0)

        macd = self._macd(px, int(self.p["fastLength"]), int(self.p["slowLength"]), int(self.p["signalLength"]))
        macz_line = (zscore * A) + (macd["lMACD"] / (macd_std * B))
        signal = macz_line.rolling(window=signalLength, min_periods=signalLength).mean()
        histmacz = macz_line - signal
        return pd.DataFrame({"histmacz": histmacz}, index=df.index)

    def compute(self, df: pd.DataFrame) -> pd.DataFrame:
        df = _ensure_price(df)
        p = self.p
        out = pd.DataFrame(index=df.index)

        sEMA = self._ema(df["Price"], int(p["sEma_Length"]))
        fEMA = self._ema(df["Price"], int(p["fEma_Length"]))
        out["EMA_longCond"] = (fEMA > sEMA) & (sEMA > sEMA.shift(1))
        out["EMA_shortCond"] = (fEMA < sEMA) & (sEMA < sEMA.shift(1))

        adx_df = self._adx_block(df, int(p["ADX_len"]),
                                 int(p.get("ADX_smo", p["ADX_len"])),
                                 float(p["th"]))
        out = out.join(adx_df)

        sar = self._sar_tv(df["high"], df["low"],
                           float(p["Sst"]), float(p["Sinc"]), float(p["Smax"]),
                           df["Price"])
        out["SAR"] = sar
        out["SAR_longCond"] = sar < df["Price"]
        out["SAR_shortCond"] = sar > df["Price"]

        macd_df = self._macd(df["Price"],
                             int(p["fastLength"]),
                             int(p["slowLength"]),
                             int(p["signalLength"]))
        out = out.join(macd_df)
        if p.get("MACD_options", "MACD") == "MAC-Z":
            out = out.join(self._macz(df))
        else:
            out["histmacz"] = out["hist"]
        out["MACD_longCond"] = out["histmacz"] > 0
        out["MACD_shortCond"] = out["histmacz"] < 0

        L = int(p["BB_Length"])
        mult = float(p["BB_mult"])
        mid = df["Price"].rolling(window=L, min_periods=L).mean()
        std = df["Price"].rolling(window=L, min_periods=L).std(ddof=0)
        bb_upper = mid + mult * std
        bb_lower = mid - mult * std
        out["BB_upper"] = bb_upper
        out["BB_lower"] = bb_lower
        out["BB_middle"] = mid
        out["BB_width"] = (bb_upper - bb_lower) / mid

        vol_sma = df["volume"].rolling(window=int(p["sma_Length"]), min_periods=1).mean()
        vol_flag = df["volume"] > vol_sma * float(p["volume_f"])
        out["VOL_shortCond"] = vol_flag
        out["VOL_longCond"] = vol_flag

        out["bbMinWidth01"] = float(p["bbMinWidth01"]) / 100.0
        out["bbMinWidth02"] = float(p["bbMinWidth02"]) / 100.0

        return out

# ======================================================
# ============= Backtester short Super 8 ===============
# ======================================================

@dataclass
class ShortParams:
    Position: str = "Both"
    TP_options: str = "Both"
    SL_options: str = "Both"
    tp: float = 3.6
    sl: float = 5.0
    atrPeriodSl: int = 21
    multiplierPeriodSl: float = 9.5
    trailOffset: float = 0.38
    reverse_exit: bool = True
    ignore_additional_entries: bool = True
    exit_on_entry_loss: bool = False
    start_time: Optional[pd.Timestamp] = None
    intrabar_touch: bool = True
    bar_path: str = "OLHC"
    no_same_bar_exit: bool = True


class Super8ShortBacktester:
    def __init__(self,
                 indicator_params: Dict[str, Any],
                 short_params: ShortParams,
                 use_spread: bool = True,
                 log_return: bool = True):
        self.indicator_params = indicator_params
        self.short_params = short_params
        self.use_spread = use_spread
        self.log_return = log_return
        self._last_bt = None

    def run(self, df: pd.DataFrame) -> pd.DataFrame:
        data = df.copy()
        if "time" in data.columns:
            data["time"] = pd.to_datetime(data["time"], utc=True, errors="coerce")
            data = data.dropna(subset=["time"]).set_index("time").sort_index()

        data = _ensure_price(data)
        for col in ["high", "low", "volume"]:
            if col not in data.columns:
                raise ValueError(f"Data missing required column '{col}'.")

        if "Return" not in data.columns:
            data["Return"] = np.log(data["Price"] / data["Price"].shift(1))

        if "Spread" not in data.columns:
            raise ValueError("Missing 'Spread' in data — compute Spread before backtesting.")

        indicators = Super8Indicators(self.indicator_params).compute(data)
        base = data.join(indicators, how="left")

        EMA_shortCond = base.get("EMA_shortCond", pd.Series(False, index=base.index)).fillna(False)
        EMA_longCond = base.get("EMA_longCond", pd.Series(False, index=base.index)).fillna(False)
        ADX_shortCond = base.get("ADX_shortCond", pd.Series(False, index=base.index)).fillna(False)
        ADX_longCond = base.get("ADX_longCond", pd.Series(False, index=base.index)).fillna(False)
        SAR_shortCond = base.get("SAR_shortCond", pd.Series(False, index=base.index)).fillna(False)
        SAR_longCond = base.get("SAR_longCond", pd.Series(False, index=base.index)).fillna(False)
        VOL_shortCond = base.get("VOL_shortCond", pd.Series(False, index=base.index)).fillna(False)
        VOL_longCond = base.get("VOL_longCond", pd.Series(False, index=base.index)).fillna(False)

        h = base.get("hist", pd.Series(0.0, index=base.index))
        MACD_shortCond = h.lt(0).fillna(False)
        MACD_longCond = h.gt(0).fillna(False)

        BB_upper = base.get("BB_upper", pd.Series(np.nan, index=base.index))
        BB_lower = base.get("BB_lower", pd.Series(np.nan, index=base.index))
        BB_middle = base.get("BB_middle", pd.Series(np.nan, index=base.index))
        BB_width = base.get("BB_width", pd.Series(np.nan, index=base.index))
        bbMin01 = base.get("bbMinWidth01", pd.Series(0.05, index=base.index))

        allow_short = (self.short_params.Position != "Long")
        BB_short01 = (allow_short
                      & (~ADX_longCond)
                      & _crossover(base["high"], BB_upper)
                      & EMA_shortCond
                      & (BB_width > bbMin01))
        BB_short01 = BB_short01.fillna(False)

        shortCond = (allow_short
                     & EMA_shortCond
                     & ADX_shortCond
                     & SAR_shortCond
                     & MACD_shortCond
                     & VOL_shortCond)
        entry_cond = (shortCond | BB_short01).fillna(False)

        exit_cond = (EMA_longCond | ADX_longCond | SAR_longCond | MACD_longCond).fillna(False)

        entry_sig = entry_cond.astype("boolean").shift(1).fillna(False).astype(bool)
        exit_sig = exit_cond.astype("boolean").fillna(False).astype(bool)

        base["DBG_exit_cond"] = exit_cond.astype(int)
        base["DBG_EMA_longCond"] = EMA_longCond.astype(int)
        base["DBG_SAR_longCond"] = SAR_longCond.astype(int)
        base["DBG_MACD_longCond"] = MACD_longCond.astype(int)

        tr = np.maximum(
            base["high"] - base["low"],
            np.maximum((base["high"] - base["Price"].shift(1)).abs(),
                       (base["low"] - base["Price"].shift(1)).abs()),
        )
        atr_rma = _rma(pd.Series(tr, index=base.index), self.short_params.atrPeriodSl)
        atr_sl_short_raw = base["high"] + atr_rma * self.short_params.multiplierPeriodSl

        _open = base["open"] if "open" in base.columns else base["Price"].shift(1).fillna(base["Price"])

        atr_sl_short_series = pd.Series(index=base.index, dtype=float)
        prev_stop = np.nan
        for i in range(len(base.index)):
            raw = float(atr_sl_short_raw.iloc[i]) if not pd.isna(atr_sl_short_raw.iloc[i]) else np.nan
            o = float(_open.iloc[i]) if not pd.isna(_open.iloc[i]) else raw
            if i == 0 or np.isnan(prev_stop) or np.isnan(raw) or np.isnan(o):
                val = raw
            else:
                val = min(raw, prev_stop) if o < prev_stop else raw
            atr_sl_short_series.iloc[i] = val
            prev_stop = val
        base["DBG_ATR_SL_Short"] = atr_sl_short_series

        if self.short_params.TP_options in ("Both", "Donchian"):
            DClower = base["low"].rolling(window=self.indicator_params["DClength"],
                                          min_periods=self.indicator_params["DClength"]).min()
        else:
            DClower = pd.Series(np.nan, index=base.index)

        position = []
        pos = 0
        entry_price = np.nan

        for t, row in base.iterrows():
            if self.short_params.start_time is not None and t < self.short_params.start_time:
                position.append(0)
                continue

            price = row["Price"]

            if pos == 0 and entry_sig.loc[t]:
                pos = -1
                entry_price = price

            avg_price = entry_price if (pos == -1 and not np.isnan(entry_price)) else price

            if self.short_params.TP_options == "Both":
                dc_val = DClower.loc[t] if 'DClower' in locals() and t in DClower.index else np.nan
                tp_level = np.nanmin([
                    dc_val if not np.isnan(dc_val) else np.inf,
                    (1.0 - self.short_params.tp / 100.0) * avg_price,
                ])
            elif self.short_params.TP_options == "Normal":
                tp_level = (1.0 - self.short_params.tp / 100.0) * avg_price
            elif self.short_params.TP_options == "Donchian":
                dc_val = DClower.loc[t] if 'DClower' in locals() and t in DClower.index else np.nan
                tp_level = dc_val if not np.isnan(dc_val) else avg_price
            else:
                tp_level = np.nan

            atr_sl_val = atr_sl_short_series.loc[t] if t in atr_sl_short_series.index else np.nan
            if self.short_params.SL_options == "Both":
                sl_level = max(atr_sl_val, (1.0 + self.short_params.sl / 100.0) * avg_price)
            elif self.short_params.SL_options == "Normal":
                sl_level = (1.0 + self.short_params.sl / 100.0) * avg_price
            elif self.short_params.SL_options == "ATR":
                sl_level = atr_sl_val
            else:
                sl_level = np.nan

            if pos == -1:
                tp_hit = (not np.isnan(tp_level)) and (price <= tp_level)
                sl_hit = (not np.isnan(sl_level)) and (price >= sl_level)
                rev_hit = bool(self.short_params.reverse_exit and exit_sig.loc[t])

                if tp_hit or sl_hit or rev_hit:
                    pos = 0
                    entry_price = np.nan

            position.append(pos)

        base["position"] = position

        base["Return"] = base["Return"].fillna(0.0)
        base["strategy"] = base["position"].fillna(0) * base["Return"]

        pos_changes = base["position"].astype(int).diff().fillna(base["position"]).abs()
        base["trades"] = np.where(pos_changes != 0, 1, 0)

        if self.use_spread:
            base["Spread"] = base["Spread"].fillna(0.0)
            base["strategy"] = base["strategy"] - base["trades"] * base["Spread"]

        if self.log_return:
            base["creturn"] = base["Return"].cumsum().apply(np.exp)
            base["cstrategy"] = base["strategy"].cumsum().apply(np.exp)
        else:
            base["creturn"] = (1.0 + base["Return"]).cumprod()
            base["cstrategy"] = (1.0 + base["strategy"]).cumprod()

        self._last_bt = base.copy()
        return base

# ======================================================
# ================== Funcție obiectiv ==================
# ======================================================

def optimal_strategy(params_tuple: tuple,
                     symbol: str,
                     start: str,
                     end: str,
                     bar_length: str = "H2") -> float:
    """
    Funcție obiectiv pentru optimizare:
    maximizează P&L cu constrângeri soft pe DD, Profit Factor și număr de tranzacții.
    """
    try:
        if symbol in all_data:
            df = all_data[symbol]
        else:
            file_path = symbol_csv_path_case_insensitive(symbol) or symbol_csv_path(symbol)
            if not Path(file_path).exists():
                return 1e6
            df = pd.read_csv(file_path, index_col="time", parse_dates=True)
            df = _ensure_price(df)
            COST_PER_SIDE = -np.log(1.0 - 0.00055)
            df["Spread"] = COST_PER_SIDE
            all_data[symbol] = df

        df_period = df.loc[start:] if not end else df.loc[start:end]
        if df_period.empty:
            return 1e6

        ind_params = indicator_params_default.copy()
        short_vals = short_params_default.copy()

        for i, name in enumerate(names):
            val = params_tuple[i]
            if name in ind_params:
                if name in ["sEma_Length", "fEma_Length", "ADX_len", "ADX_smo",
                            "fastLength", "slowLength", "signalLength",
                            "sma_Length", "BB_Length", "DClength"]:
                    ind_params[name] = max(1, int(round(val)))
                else:
                    ind_params[name] = float(val)
            else:
                if name == "atrPeriodSl":
                    short_vals[name] = max(1, int(round(val)))
                else:
                    short_vals[name] = float(val)

        ind_params["MACD_options"] = indicator_params_default["MACD_options"]
        ind_params["lengthz"] = indicator_params_default["lengthz"]
        ind_params["lengthStdev"] = indicator_params_default["lengthStdev"]
        ind_params["A"] = indicator_params_default["A"]
        ind_params["B"] = indicator_params_default["B"]
        ind_params["bbMinWidth02"] = indicator_params_default["bbMinWidth02"]

        if not is_param_combo_valid(ind_params, short_vals):
            return 1e6

        sl_opt = short_vals.get("SL_options", short_params_default.get("SL_options", "Both"))
        short_obj = ShortParams(
            Position=short_vals.get("Position", short_params_default.get("Position", "Short")),
            TP_options="Both",
            SL_options=sl_opt,
            tp=short_vals["tp"],
            sl=short_vals["sl"],
            atrPeriodSl=short_vals["atrPeriodSl"],
            multiplierPeriodSl=short_vals["multiplierPeriodSl"],
            trailOffset=short_vals["trailOffset"],
        )
        backtester = Super8ShortBacktester(
            indicator_params=ind_params,
            short_params=short_obj,
            use_spread=True,
            log_return=True,
        )
        bt = backtester.run(df_period)
        if bt.empty:
            return 1e6

        perf = float(bt["cstrategy"].iloc[-1])
        if perf <= 0:
            return 1e6

        metrics = compute_performance_metrics(bt, initial_equity=1.0, log_returns=backtester.log_return)
        dd = float(metrics["max_drawdown_pct"]) / 100.0
        pf = float(metrics["profit_factor"])
        trades = int(metrics["total_trades"])

        R = float(np.log(perf))

        dd_over = max(0.0, dd - 0.15)
        pen_dd = 50.0 * dd_over ** 2

        pf_eff = min(pf, 5.0)
        pf_under = max(0.0, 1.5 - pf_eff)
        pen_pf = 1.0 * pf_under ** 2

        if trades < 50:
            pen_tr = 1.0 * ((50 - trades) / 50.0) ** 2
        elif trades > 600:
            pen_tr = 1.0 * ((trades - 600) / 600.0) ** 2
        else:
            pen_tr = 0.0

        penalty = pen_dd + pen_pf + pen_tr
        score = R - penalty

        return -score

    except Exception:
        # set invalid brutal ca să fie ignorat
        return 1e6

# ======================================================
# ============== Optimizare paralelă ===================
# ======================================================

def optimize_strategy_parallel(symbol, start_par, bnds, start, end, bar_length,
                               max_workers=4, seed=None):
    rng = np.random.default_rng(seed)

    def rand_start():
        return tuple(rng.uniform(lo, hi) for (lo, hi) in bnds)

    best = None
    starts = [start_par] + [rand_start() for _ in range(max_workers - 1)]

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = [
            ex.submit(
                minimize,
                optimal_strategy,
                s,
                args=(symbol, start, end or "", bar_length),
                method="Powell",
                bounds=bnds,
                options={"maxiter": 220, "xtol": 1e-2, "ftol": 1e-2},
            )
            for s in starts
        ]

        for f in as_completed(futs):
            try:
                r = f.result()
                if r.success and (best is None or r.fun < best.fun):
                    best = r
            except Exception as e:
                print("Eroare în calcul:", e)
    return best

# ======================================================
# ================== Data management ===================
# ======================================================

def manage_symbol_data(symbol: str, start: str, end: str, file_path: str):
    if Path(file_path).resolve().exists():
        return
    alt_path = symbol_csv_path_case_insensitive(symbol)
    if alt_path is not None:
        return
    raise FileNotFoundError(
        f"Fișierul de date pentru {symbol} nu există: "
        f"'{file_path}' (sau '{os.path.join(DATA_DIR, symbol + '.CSV')}').\n"
        f"Setează DATA_DIR corect sau exportă SUPER8_DATA_DIR către folderul cu CSV-uri."
    )

# ======================================================
# ==================== Main runner =====================
# ======================================================

def main(symbols: list) -> pd.DataFrame:
    start_date = "2021-06-01"

    df_results = pd.DataFrame(columns=[
        "Symbol", "Parameters", "Performance", "Trades",
        "TotalPnL", "TotalPnL_pct", "MaxDrawdown_pct",
        "WinRate_pct", "ProfitFactor", "MaxTradeDuration_days",
    ])

    for symbol in symbols:
        file_path = symbol_csv_path(symbol)
        try:
            manage_symbol_data(symbol, start_date, "", file_path)
            resolved_path = symbol_csv_path_case_insensitive(symbol) or file_path
        except Exception as e:
            print(f"Nu s-au putut obține datele pentru {symbol}: {e}")
            continue

        try:
            print(f"[{symbol}] Citim datele din: {resolved_path}")
            data = pd.read_csv(resolved_path, index_col="time", parse_dates=True)
        except Exception as e:
            print(f"Eroare la citirea fișierului pentru {symbol}: {e}")
            continue

        try:
            data = _ensure_price(data)
            COST_PER_SIDE = -np.log(1.0 - 0.00055)
            data["Spread"] = COST_PER_SIDE
        except Exception as e:
            print(f"[{symbol}] Eroare la pregătirea datelor (Price/Spread): {e}")
            continue

        all_data[symbol] = data

        last_idx = data.index.max()
        last_date_str = last_idx.strftime("%Y-%m-%d") if pd.notna(last_idx) else ""
        attempt = 0
        max_attempts = 4
        success = False
        last_exception = None

        while attempt < max_attempts and not success:
            try:
                print(f"Optimizare pentru simbolul {symbol} pe perioada {start_date} - {last_date_str}")
                opt_result = optimize_strategy_parallel(symbol, start_params, bounds, start_date, "", "H2")
                if opt_result is None or not opt_result.success:
                    raise ValueError("Optimizarea a eșuat.")

                best_params = opt_result.x
                output_params = []
                for i, name in enumerate(names):
                    val = best_params[i]
                    if name in ["sEma_Length", "fEma_Length", "ADX_len", "ADX_smo",
                                "fastLength", "slowLength", "signalLength",
                                "sma_Length", "BB_Length", "DClength", "atrPeriodSl"]:
                        v = max(1, int(round(val)))
                    else:
                        v = round(float(val), 3)
                    output_params.append(v)

                ind_params_opt = indicator_params_default.copy()
                short_vals_opt = short_params_default.copy()
                for i, name in enumerate(names):
                    if name in ind_params_opt:
                        if name in ["sEma_Length", "fEma_Length", "ADX_len", "ADX_smo",
                                    "fastLength", "slowLength", "signalLength",
                                    "sma_Length", "BB_Length", "DClength"]:
                            ind_params_opt[name] = max(1, int(round(best_params[i])))
                        else:
                            ind_params_opt[name] = float(best_params[i])
                    else:
                        if name == "atrPeriodSl":
                            short_vals_opt[name] = max(1, int(round(best_params[i])))
                        else:
                            short_vals_opt[name] = float(best_params[i])

                ind_params_opt["MACD_options"] = indicator_params_default["MACD_options"]
                ind_params_opt["lengthz"] = indicator_params_default["lengthz"]
                ind_params_opt["lengthStdev"] = indicator_params_default["lengthStdev"]
                ind_params_opt["A"] = indicator_params_default["A"]
                ind_params_opt["B"] = indicator_params_default["B"]
                ind_params_opt["bbMinWidth02"] = indicator_params_default["bbMinWidth02"]

                if not is_param_combo_valid(ind_params_opt, short_vals_opt):
                    print(f"Set de parametri invalid pentru {symbol}: {why_invalid(ind_params_opt, short_vals_opt)} (ignorat).")
                    break

                sl_opt_full = short_vals_opt.get("SL_options", short_params_default.get("SL_options", "Both"))
                short_obj_opt = ShortParams(
                    Position="Both",
                    TP_options="Both",
                    SL_options=sl_opt_full,
                    tp=short_vals_opt["tp"],
                    sl=short_vals_opt["sl"],
                    atrPeriodSl=short_vals_opt["atrPeriodSl"],
                    multiplierPeriodSl=short_vals_opt["multiplierPeriodSl"],
                    trailOffset=short_vals_opt["trailOffset"],
                )
                backtester_opt = Super8ShortBacktester(
                    indicator_params=ind_params_opt,
                    short_params=short_obj_opt,
                    use_spread=True,
                    log_return=True,
                )
                bt_full = backtester_opt.run(data.loc[start_date:])

                final_perf = float(bt_full["cstrategy"].iloc[-1]) if not bt_full.empty else 1.0
                pos_series = bt_full["position"]
                entry_events = (pos_series != 0) & (pos_series.shift(1).fillna(0) == 0)
                num_trades = int(entry_events.sum())

                metrics = compute_performance_metrics(bt_full, initial_equity=1.0,
                                                      log_returns=backtester_opt.log_return)
                print_performance_summary(symbol, metrics)

                param_display = {name: output_params[i] for i, name in enumerate(names)}
                print(f"Finalizat optimizarea pentru {symbol}.", flush=True)
                for pname, pval in param_display.items():
                    print(f"  {pname}: {pval}", flush=True)
                print(f"  Performance: {final_perf:.4f} | Trades: {num_trades}", flush=True)

                new_row = {
                    "Symbol": symbol,
                    "Parameters": output_params,
                    "Performance": final_perf,
                    "Trades": num_trades,
                    "TotalPnL": metrics.get("total_pnl"),
                    "TotalPnL_pct": metrics.get("total_pnl_pct"),
                    "MaxDrawdown_pct": metrics.get("max_drawdown_pct"),
                    "WinRate_pct": metrics.get("win_rate_pct"),
                    "ProfitFactor": metrics.get("profit_factor"),
                    "MaxTradeDuration_days": metrics.get("max_trade_duration_days"),
                }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
                success = True

            except (ConnectionError, TimeoutError, OSError) as ex:
                last_exception = ex
                print(f"Eroare temporară la simbolul {symbol}: {ex}. Se reîncearcă.")
            except Exception as ex:
                last_exception = ex
                print(f"Eroare la simbolul {symbol}: {ex}. Trecem la următorul simbol.")
                break
            finally:
                attempt += 1
                if not success:
                    print(f"Încercare: {attempt}")
                    if attempt >= max_attempts:
                        print(f"Procesul pentru simbolul {symbol} a eșuat: număr maxim de încercări atins.")
                    elif isinstance(last_exception, (ConnectionError, TimeoutError, OSError)):
                        wait_time = 5 * attempt
                        print(f"Așteptăm {wait_time} secunde înainte de următoarea încercare.")
                        time.sleep(wait_time)

    if not df_results.empty:
        df_results = df_results.sort_values(by="Performance", ascending=False, ignore_index=True)
    return df_results

# ======================================================
# ==================== Entry point =====================
# ======================================================

if __name__ == "__main__":
    symbols = ["AVAXUSDT"]
    start_time = time.time()
    results_df = main(symbols)
    end_time = time.time()
    print(f"Timpul total de execuție: {end_time - start_time:.2f} secunde")
    print(results_df)
    # results_df.to_csv("rezultate_optimizare.csv", index=False)


Optimizez 3 parametri: ['atrPeriodSl', 'multiplierPeriodSl', 'sl']
[AVAXUSDT] Citim datele din: C:\Users\Stefan\Desktop\Curs Trading\Strategii\Super8\Optimizare Parametri\Simboluri_Binance\AVAXUSDT.csv
Optimizare pentru simbolul AVAXUSDT pe perioada 2021-06-01 - 2025-12-01
